# Polifonia web portal data layer

## Imports and data parsing

In [ ]:
import re
from collections import defaultdict
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd

In [ ]:
# Parse google spreadsheet

# authenticate
auth.authenticate_user()

# spreadsheet: Research topics
gc = gspread.authorize(GoogleCredentials.get_application_default())
spreadsheet = gc.open('Polifonia_web_portal_CQ')
topics = spreadsheet.get_worksheet(1)

# Transform tables to dataframes. 

def make_header(df):
  new_header = df.iloc[0] 
  df = df[1:] 
  df.columns = new_header
  return df

# CQs
rows = topics.get_all_values()
df = pd.DataFrame.from_records(rows)
df = make_header(df)

persona = [re.split('\d+',values)[0] for values in df['CQ ID']]    
df["persona"] = persona
df.head()

KeyboardInterrupt: ignored

## Overview 

 - CQ classification
 - CQ addressing only main entities/properties
 - Personas coverage

In [ ]:
all = len(df)
biblio_data = len(df[df["Bibl. \ndata"] == 'yes'])
music_data = len(df[df["Music \ndata"] == 'yes'])
ling_data = len(df[df["Linguistic data"] == 'yes'])

biblio_data_only = len(df[(df["Bibl. \ndata"] == 'yes') & (df["Music \ndata"] == 'no') & (df["Linguistic data"] == 'no')])
music_data_only = len(df[(df["Music \ndata"] == 'yes') & (df["Bibl. \ndata"] == 'no') & (df["Linguistic data"] == 'no')])
ling_data_only = len(df[(df["Linguistic data"] == 'yes') & (df["Music \ndata"] == 'no') & (df["Bibl. \ndata"] == 'no')])

biblio_ling = len(df[(df["Bibl. \ndata"] == 'yes') & (df["Linguistic data"] == 'yes')])
biblio_music = len(df[(df["Bibl. \ndata"] == 'yes') & (df["Music \ndata"] == 'yes')])
biblio_music_ling = len(df[(df["Bibl. \ndata"] == 'yes') & (df["Music \ndata"] == 'yes')& (df["Linguistic data"] == 'yes')])

music_ling = len(df[(df["Music \ndata"] == 'yes')& (df["Linguistic data"] == 'yes')])
others = len(df[(df["Bibl. \ndata"] == 'no') & (df["Music \ndata"] == 'no')& (df["Linguistic data"] == 'no')])

print("Total",all,\
      "\n - Bibliographic",biblio_data, "(bibl. only",biblio_data_only,"; linguistic",biblio_ling,"; music",biblio_music,"; both", biblio_music_ling,")"\
      "\n - Music",music_data, "(music only",music_data_only,"; bibliographic",biblio_music,"; music",music_ling ,"; both", biblio_music_ling,")"\
      "\n - Linguistic",ling_data, "(ling only",ling_data_only,"; bibliographic",biblio_ling,"; music",music_ling ,"; both", biblio_music_ling,")"\
      "\n - Other",others)

to_include = df[df["Include\nin portal?"] == 'yes']
to_include_bibl = len(to_include[to_include["Bibl. \ndata"] == 'yes'])
to_include_music = len(to_include[to_include["Music \ndata"] == 'yes'])
to_include_ling = len(to_include[to_include["Linguistic data"] == 'yes'])

print("\nTo be included in indexes (based on web portal data layer)", len(to_include), "(",len(to_include)*100/all,"%)"\
      "\n - Bibliographic",to_include_bibl,\
      "\n - Music", to_include_music,\
      "\n - Linguistic",to_include_ling)

pf = to_include
pf_persona = pf.groupby("persona")["CQ ID"].count()
df_persona = df.groupby("persona")["CQ ID"].count()
personas = pd.merge(pf_persona, df_persona, on="persona")
personas.reset_index(level=0, inplace=True)
personas["percentage"] = (personas["CQ ID_x"].astype(int)*100/personas["CQ ID_y"].astype(int)).apply(lambda x: f"{x:.2f}")
personas['_'] = personas[['persona', 'percentage']].apply(tuple, axis=1)

print("\nPersonas covered in indexes")
for p,c in personas['_']:
  print(" - ",p,c,"%")

pf_list = pf_persona.index.tolist()
df_list = df_persona.index.tolist()
diff = list(set(df_list) - set(pf_list))

print("\nPersonas not covered in indexes:\n -", "\n - ".join(diff))


Total 217 
 - Bibliographic 156 (bibl. only 81 ; linguistic 62 ; music 23 ; both 10 )
 - Music 66 (music only 38 ; bibliographic 23 ; music 15 ; both 10 )
 - Linguistic 80 (ling only 13 ; bibliographic 62 ; music 15 ; both 10 )
 - Other 3

To be included in indexes (based on web portal data layer) 47 ( 21.658986175115206 %)
 - Bibliographic 47 
 - Music 5 
 - Linguistic 6

Personas covered in indexes
 -  Carolina 22.73 %
 -  David 23.53 %
 -  Keith 66.67 %
 -  Keoma 27.27 %
 -  Mark 31.82 %
 -  Ortenz 15.91 %
 -  Patrizia 12.50 %
 -  Sonia 55.56 %
 -  William 57.14 %

Personas not covered in indexes:
 - Sethus
 - Sofia
 - Anna


## High priority entities / properties for Bibliographic CQs

In [ ]:
print("\nEntities/properties covered in indexes")

entities = set(to_include["Main \nentities"].tolist())
entities = list(set([i if ';' not in i else y for i in entities for y in i.split("; ") ]))
entities_properties = defaultdict(set)
for e in entities:
  if ':' in e:
    entity, properties = e.split(":")[0], e.split(":")[1].split(",")
    for p in properties:
      entities_properties[entity].add(p)

for k,v in entities_properties.items():
  print("\n - ",k+":", ",".join(p for p in v))


Entities/properties covered in indexes

 -  Instrument:  builder, type, date, place of production, material

 -  Musical performance:  mediums of performance collection, place, date, all, musical composition, composition, place of production, performer

 -  Music work:  genre, place of production, date, all, composer, title

 -  Source:  creator, type, place of production

 -  Performer:  role, medium of performance


## Low priority entities / properties for Bibliographic CQs

In [ ]:
print("\nEntities/properties to be added ASAP in web portal data layer")
new_entities = set(to_include["Additional \nentities/props"].tolist())
new_entities = list(set([i if ';' not in i else y for i in new_entities for y in i.split("; ") ]))
relevant_personas = to_include[to_include["Additional \nentities/props"].str.contains(new_entities[1])]
print(" -", ",".join(new_entities), "\nRelevant to CQs:\n -",", ".join(relevant_personas["CQ ID"].tolist()))

# CQs that cover all bibliographic data but where music/linguistic data are excluded
biblio_to_extend = df[(df["Bibl. \ndata"] == 'yes') & (df["Music \ndata"] == 'no') \
            & (df["Linguistic data"] == 'no') \
            & (df["Linguistic data"] == 'no')\
            & (df["Include\nin portal?"] == 'no')]

print("\nTo include all CQs relevant to bibliographic data only (",len(biblio_to_extend),'- for a total of',len(biblio_to_extend)+len(to_include),'out of',all,'(',(len(biblio_to_extend)+len(to_include))*100/all,'%)), we\'d need to add')
new_entities = set(biblio_to_extend["Additional \nentities/props"].tolist())
new_entities = list(set([i if ';' not in i else y for i in new_entities for y in i.split("; ") ]))
new_entities_properties = defaultdict(set)
for e in new_entities:
    entity = e.split(":")[0]
    properties = e.split(":")[1].split(",") if ':' in e else e
    name = entity if ':' in e else "_"+entity
    if isinstance(properties,list):
      for p in properties:
        new_entities_properties[name].add(p)
    else:
      new_entities_properties[name].add('')
for k,v in new_entities_properties.items():
  print("\n - ",k+":", ",".join(p for p in v))

# all bibliographic data (1) to be included, and (2) not to be included that focus on bibl. data only
all_biblio = pd.concat([biblio_to_extend,to_include]).drop_duplicates().reset_index(drop=True)
all_biblio_persona = all_biblio.groupby("persona")["CQ ID"].count()
df_persona = df.groupby("persona")["CQ ID"].count()
personas = pd.merge(all_biblio_persona, df_persona, on="persona")
personas.reset_index(level=0, inplace=True)
personas["percentage"] = (personas["CQ ID_x"].astype(int)*100/personas["CQ ID_y"].astype(int)).apply(lambda x: f"{x:.2f}")
personas['_'] = personas[['persona', 'percentage']].apply(tuple, axis=1)

print("\nPersonas covered in indexes")
for p,c in personas['_']:
  print(" - ",p,c,"%")

all_biblio_list = all_biblio_persona.index.tolist()
df_list = df_persona.index.tolist()
diff = list(set(df_list) - set(all_biblio_list))

print("\nPersonas not covered in indexes:\n -", "\n - ".join(diff))




Entities/properties to be added ASAP in web portal data layer
 - ,_vague 
Relevant to CQs:
 - Keith1-CQ1

To include all CQs relevant to bibliographic data only ( 40 - for a total of 87 out of 217 ( 40.09216589861751 %)), we'd need to add

 -  _: 

 -  __vague: 

 -  Musical performance:  hasPerformerSituation, date, building (subproperty of place)

 -  Performer:  occupation, places, employer, birthplace

 -  Collection:  composition, all

 -  Musical work:  lyrics

 -  Instrument:  current place, chromatic range,  chromatic range

 -  Source: date

 -  PerformerSituation:  hasInstrumentOrVoice

 -  _new property to describe 'kind' of repertoire: 

Personas covered in indexes
 -  Carolina 50.00 %
 -  David 35.29 %
 -  Keith 66.67 %
 -  Keoma 72.73 %
 -  Mark 63.64 %
 -  Ortenz 27.27 %
 -  Patrizia 25.00 %
 -  Sethus 9.68 %
 -  Sofia 16.00 %
 -  Sonia 66.67 %
 -  William 92.86 %

Personas not covered in indexes:
 - Anna


## Low priority entities / properties for mixed CQs

In [ ]:
# CQs covering mixed topics currently set as not to be included
biblio_all = df[(df["Bibl. \ndata"] == 'yes') & (df["Include\nin portal?"] == 'no')]
biblio_mixed_only = biblio_all[(biblio_all["Music \ndata"] == 'yes') | (biblio_all["Linguistic data"] == 'yes')]
len(biblio_mixed_only)


print("\nCQs relevant to mixed topics including bibl. data (",len(biblio_mixed_only))
new_entities = set(biblio_mixed_only["Additional \nentities/props"].tolist())
new_entities = list(set([i if ';' not in i else y for i in new_entities for y in i.split("; ") ]))
new_entities_properties = defaultdict(set)
for e in new_entities:
    entity = e.split(":")[0]
    properties = e.split(":")[1].split(",") if ':' in e else e
    name = entity if ':' in e else "_"+entity
    if isinstance(properties,list):
      for p in properties:
        new_entities_properties[name].add(p)
    else:
      new_entities_properties[name].add('')
for k,v in new_entities_properties.items():
  print("\n - ",k+":", ",".join(p for p in v))

# all bibliographic data (1) to be included, and (2) not to be included that focus on bibl. data only
mixed_biblio_persona = biblio_mixed_only.groupby("persona")["CQ ID"].count()
df_persona = df.groupby("persona")["CQ ID"].count()
personas = pd.merge(mixed_biblio_persona, df_persona, on="persona")
personas.reset_index(level=0, inplace=True)
personas["percentage"] = (personas["CQ ID_x"].astype(int)*100/personas["CQ ID_y"].astype(int)).apply(lambda x: f"{x:.2f}")
personas['_'] = personas[['persona', 'percentage']].apply(tuple, axis=1)

print("\nPersonas covered in indexes")
for p,c in personas['_']:
  print(" - ",p,c,"%")

all_biblio_list = mixed_biblio_persona.index.tolist()
df_list = df_persona.index.tolist()
diff = list(set(df_list) - set(all_biblio_list))

print("\nPersonas not covered in indexes:\n -", "\n - ".join(diff))
# count how many are still vague without any main entities
print("too vague:")
len(biblio_mixed_only[(biblio_mixed_only["Main \nentities"] == '') & (biblio_mixed_only["Additional \nentities/props"] == '_vague')] )


CQs relevant to mixed topics including bibl. data ( 69

 -  _: 

 -  __vague: 

 -  Source: hasSubject, hasSubject, date, audience, listener

 -  Music work:  text, style

 -  Music feature:  parent work

 -  Musical performance:  type, hasPerformerSituation, subject of, date, audience, commissioner

 -  PerformerSituation:   hasInstrumentOrVoice, hasInstrumentOrVoice

 -  Performer:  relatedTo

 -  Musical work:  lyrics

 -  Collection:  composition

Personas covered in indexes
 -  Carolina 40.91 %
 -  David 23.53 %
 -  Keoma 18.18 %
 -  Mark 27.27 %
 -  Ortenz 65.91 %
 -  Patrizia 62.50 %
 -  Sethus 16.13 %
 -  Sofia 28.00 %
 -  Sonia 11.11 %

Personas not covered in indexes:
 - Keith
 - William
 - Anna
too vague:


11

Download data

In [ ]:
df['coverage'] = 'none'
df.loc[(df["Bibl. \ndata"] == 'yes') & (df["Music \ndata"] == 'no') & (df["Linguistic data"] == 'no'), 'coverage'] = 'full' 
df.loc[(df["Bibl. \ndata"] == 'yes') & ((df["Music \ndata"] == 'yes') | (df["Linguistic data"] == 'yes')), 'coverage'] = 'partial'  
from google.colab import files
df.to_csv('df.csv')
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>